In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier
import pickle
from sklearn.externals import joblib
import os

In [2]:
train = pd.read_csv('trainms.csv') 
test = pd.read_csv('testms.csv')

In [3]:
test['treatment'] = pd.Series(['Yes']*259)

In [4]:
train = pd.concat([train, test])

C:\Users\Hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [5]:
train.shape

(1259, 28)

In [6]:
train.head()

,Age,Country,Gender,Timestamp,anonymity,benefits,care_options,comments,coworkers,family_history,...,remote_work,s.no,seek_help,self_employed,state,supervisor,tech_company,treatment,wellness_program,work_interfere
0,37,United States,Female,2014-08-27 11:29:31,Yes,Yes,Not sure,NaN,Some of them,No,...,No,1,Yes,NaN,IL,Yes,Yes,Yes,No,Often
1,44,United States,M,2014-08-27 11:29:37,Don't know,Don't know,No,NaN,No,No,...,No,2,Don't know,NaN,IN,No,No,No,Don't know,Rarely
2,32,Canada,Male,2014-08-27 11:29:44,Don't know,No,No,NaN,Yes,No,...,No,3,No,NaN,NaN,Yes,Yes,No,No,Rarely
3,31,United Kingdom,Male,2014-08-27 11:29:46,No,No,Yes,NaN,Some of them,Yes,...,No,4,No,NaN,NaN,No,Yes,Yes,No,Often
4,31,United States,Male,2014-08-27 11:30:22,Don't know,Yes,No,NaN,Some of them,No,...,Yes,5,Don't know,NaN,TX,Yes,Yes,No,Don't know,Never


In [7]:
train.isnull().sum()

Age                             0
Country                         0
Gender                          0
Timestamp                       0
anonymity                       0
benefits                        0
care_options                    0
comments                     1095
coworkers                       0
family_history                  0
leave                           0
mental_health_consequence       0
mental_health_interview         0
mental_vs_physical              0
no_employees                    0
obs_consequence                 0
phys_health_consequence         0
phys_health_interview           0
remote_work                     0
s.no                            0
seek_help                       0
self_employed                  18
state                         515
supervisor                      0
tech_company                    0
treatment                       0
wellness_program                0
work_interfere                264
dtype: int64

In [8]:
train['obs_consequence'].value_counts()

No     1075
Yes     184
Name: obs_consequence, dtype: int64

In [9]:
train['self_employed'].value_counts()

No     1095
Yes     146
Name: self_employed, dtype: int64

In [10]:
train['work_interfere'].value_counts()

Sometimes    465
Never        213
Rarely       173
Often        144
Name: work_interfere, dtype: int64

In [11]:
train['no_employees'].value_counts()

6-25              290
26-100            289
More than 1000    282
100-500           176
1-5               162
500-1000           60
Name: no_employees, dtype: int64

In [12]:
train.drop(['comments'], axis=1, inplace=True)

In [13]:
train['self_employed'] = train['self_employed'].fillna('No')

In [14]:
train['work_interfere'] = train['work_interfere'].fillna('U')

In [15]:
train['region'] = train['state']
train['region'] = train['region'].fillna("West")

west = ["WA", "OR", "CA", "NV", "ID", "MT", "WY", "UT", "AZ", "NM", "CO"]
midwest = ["ND", "SD", "NE", "KS", "MN", "IA", "MO", "WI", "IL", "IN", "OH", "MI"]
northeast = ["ME",  "NH", "VT", "MA", "CT", "RI", "NY", "PA", "NJ"]
south = ["MD", "DE", "DC", "WV",  "VA", "NC","SC", "GA", "FL", "KY", "TN", "AL", "MS", "AR", "LA", "OK", "TX"]

train['region'] = train['region'].apply(lambda x:"West" if x in west else x)
train['region'] = train['region'].apply(lambda x:"Midwest" if x in midwest else x)
train['region'] = train['region'].apply(lambda x:"Northeast" if x in northeast else x)
train['region'] = train['region'].apply(lambda x:"South" if x in south else x)

In [16]:
train.drop(['state'], axis=1, inplace=True)

In [17]:
train.drop(['Timestamp'], axis=1, inplace=True)

In [18]:
train.drop(['s.no'], axis=1, inplace=True)

In [19]:
train.head()

,Age,Country,Gender,anonymity,benefits,care_options,coworkers,family_history,leave,mental_health_consequence,...,phys_health_consequence,phys_health_interview,remote_work,seek_help,self_employed,supervisor,tech_company,treatment,wellness_program,work_interfere
0,37,United States,Female,Yes,Yes,Not sure,Some of them,No,Somewhat easy,No,...,No,Maybe,No,Yes,No,Yes,Yes,Yes,No,Often
1,44,United States,M,Don't know,Don't know,No,No,No,Don't know,Maybe,...,No,No,No,Don't know,No,No,No,No,Don't know,Rarely
2,32,Canada,Male,Don't know,No,No,Yes,No,Somewhat difficult,No,...,No,Yes,No,No,No,Yes,Yes,No,No,Rarely
3,31,United Kingdom,Male,No,No,Yes,Some of them,Yes,Somewhat difficult,Yes,...,Yes,Maybe,No,No,No,No,Yes,Yes,No,Often
4,31,United States,Male,Don't know,Yes,No,Some of them,No,Don't know,No,...,No,Yes,Yes,Don't know,No,Yes,Yes,No,Don't know,Never


In [20]:
print([w for w in train['Gender'] if re.search('^((.*(F|f)(ema..)?.*)|(.*(W|w)(oman).*))$', w)])

['Female', 'Female', 'Female', 'female', 'female', 'Female', 'female', 'Female', 'Female', 'Female', 'Female', 'Trans-female', 'Cis Female', 'female', 'F', 'F', 'female', 'Female', 'Female', 'Female', 'Female', 'F', 'F', 'Woman', 'Female', 'Female', 'Female', 'F', 'f', 'Female', 'Female', 'Female', 'female', 'Female', 'Female', 'female', 'Female', 'F', 'f', 'female', 'F', 'female', 'F', 'Female', 'f', 'Female', 'Female', 'Female', 'female', 'Female', 'Female', 'Female', 'Female', 'F', 'Female', 'f', 'female', 'F', 'f', 'Female', 'female', 'Female', 'F', 'f', 'F', 'Female', 'Femake', 'F', 'F', 'F', 'female', 'Female', 'woman', 'F', 'female', 'female', 'Female', 'female', 'female', 'female', 'female', 'female', 'Female', 'F', 'fluid', 'Female', 'Female', 'Female', 'F', 'female', 'female', 'female', 'F', 'Female ', 'Female', 'f', 'Female', 'female', 'Female', 'Female', 'Female', 'Female', 'female', 'female', 'Female', 'Female', 'f', 'Female', 'F', 'Female', 'Female', 'female', 'Female', '

In [21]:
#train['Gender'] = train['Gender'].map(lambda w: 1 if re.search('^((.*(F|f)(emale)?.*)|(.*(W|w)(oman).*))$', w) else 0)

male = ["m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]
trans = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]           
female = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail"]

train['Gender'] = train['Gender'].apply(lambda x:"male" if x in male else x)
train['Gender'] = train['Gender'].apply(lambda x:"trans" if x in trans else x)
train['Gender'] = train['Gender'].apply(lambda x:"female" if x in female else x)

In [22]:
train['leave'].replace(['Very easy', 'Somewhat easy', "Don\'t know", 'Somewhat difficult', 'Very difficult'], [1, 2, 3, 4, 5],inplace=True) 
train['work_interfere'].replace(['Never','Rarely','Sometimes','Often'],[1,2,3,4],inplace=True)
train.loc[train['no_employees']=='1-5',['no_employees']]=1
train.loc[train['no_employees']=='6-25',['no_employees']]=2
train.loc[train['no_employees']=='26-100',['no_employees']]=3
train.loc[train['no_employees']=='100-500',['no_employees']]=4
train.loc[train['no_employees']=='500-1000',['no_employees']]=5
train.loc[train['no_employees']=='More than 1000',['no_employees']]=6

In [23]:
s = int(train['Age'].median())
train['Age'] = train['Age'].apply(lambda x: s if x<18 or x>120 else x)
scaler = MinMaxScaler()
train['Age'] = scaler.fit_transform(train[['Age']])

In [25]:
X = train.drop(['treatment'],axis=1)
y = train['treatment']
X = pd.get_dummies(X)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [26]:
X_train = X[:1000]
y_train = y[:1000]
X_test = X[1000:]

In [27]:
def random_forest_classifier(features, target):
    clf = RandomForestClassifier(n_estimators=100, random_state=921)
    clf.fit(features, target)
    return clf

In [28]:
trained_model = random_forest_classifier(X_train, y_train)
predictions = trained_model.predict(X_test)
prob = trained_model.predict_proba(X_test)
P1 = []
for v in prob :
    P1.append(v[1])
prob = pd.DataFrame({'Probability_1': P1})    

In [29]:
results = pd.DataFrame({'s.no': X_test.index, 'treatment': predictions})
results['s.no'] = results['s.no'].map(lambda s: s+1)
results['treatment'] = results['treatment'].map(lambda s: 'Yes' if s==1 else 'No')
print(results.shape)

(259, 2)


In [30]:
results.to_csv('solutions.csv', index=False)